In [ ]:
from montecarlo import Die
from montecarlo import Game
from montecarlo import Analyzer

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

fair_coin = Die(["H","T)]

unfair_coin = Die(["H","T"])
unfair_coin.set_weight("H",5)

Number_of_flips = 1000

Fair_coin_set = [fair_coin, fair_coin, fair_coin]
Fair_game = Game(fair_coin_set)
Fair_game.play(number_of_flips)

Fair_game_analyzer = Analyzer(fair_game)
Fair_game_analyzer.get_face_counts()
Fair_game_analyzer.get_combos()

Fair_relative_frequency = fair_game_analyzer.get_jackpots()/number_of_flips

Value_df = pd.DataFrame({"fair":[fair_relative_frequency], "unfair": [unfair_relative_frequency]})

#Graph
Value_df.plot.bar()
Plt.xlabel("Fairness")
Plt.ylabel("Relative Frequency")
Plt.title("Scenario 1: 2-headed coin")
